Rati Patel
CS634
Prof Abduallah
December 4, 2021

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
winedf = pd.read_csv('winequality-red.csv',sep=";")
winedf.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
winedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [4]:
#no null values found in dataframe
winedf.isnull().values.all()

False

In [5]:
def calc(labs, pred):
    cm = confusion_matrix(labs,pred)
    fp = int((cm.sum(axis=0) - np.diag(cm)).sum() ) 
    fn = int((cm.sum(axis=1) - np.diag(cm)).sum() )
    tp = int(np.diag(cm).sum())
    tn = int(abs(((cm.ravel().sum())*(cm.shape[1])) - (fp + fn + tp)))
    posi = tp + fn
    negi = tn +fp
    tpr = tp/posi
    tnr = tn/negi
    fpr= fp/negi
    fnr = fn / posi
    preci = tp/(tp+fp)
    f1 = (2 *tp)/(2 * tp + fp + fn)    
    acc = (tp+tn)/(posi+negi)
    err = (fp+fn)/(posi + negi)
    bacc = (tpr+tnr)/2
    tss = (tp/(tp+fn))-(fp/(fp+tn))
    hss = (2*((tp*tn)-(fp*fn))/((tp+fn)*(fn+tn)+(tp+fp)*(fp+tn)))
    indval = [fp,fn,tp,tn,posi,negi,tpr,tnr,fpr,fnr,preci,f1,acc,err,bacc,tss,hss]
    return indval

In [6]:
def aveCalc(df):
    rst=[]
    for ind,row in df.iterrows():
        val=(row.sum()/len(row))
        rst.append(val)
    return rst

In [7]:
X= winedf.drop('quality', axis=1)
y = winedf['quality']

In [8]:
k = 10
kf = KFold(n_splits=k, random_state=None)
fc = 0
ind = ['FP','FN','TP','TN','Positive','Negative','TPR','TNR','FPR','FNR','Precision','F1','Accuracy','Error','BACC','TSS','HSS']

rf_df = pd.DataFrame(index=ind)
svm_df = pd.DataFrame(index=ind)
gru_df = pd.DataFrame(index=ind)

# Random Forest & SVM

In [9]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=50, random_state=8)

In [10]:
from sklearn.svm import SVC
model_svm = SVC()

In [11]:
fc = 0
for train_ind , test_ind in kf.split(X):
    fc=fc+1
    cn= 'fold '+str(fc)
    X_train , X_test = X.iloc[train_ind,:],X.iloc[test_ind,:]
    y_train , y_test = y[train_ind] , y[test_ind]
   
    #Random Forest
    classifier.fit(X_train,y_train)
    RF_pred = classifier.predict(X_test)

    Rf_cal = calc(y_test, RF_pred)
    rf_df[cn]=Rf_cal


  # SVM 
    model_svm.fit(X_train, y_train)
    SVM_pred = model_svm.predict(X_test)
    
    svm_cal = calc(y_test, SVM_pred)
    svm_df[cn]=svm_cal
     

rf_df['Average']=aveCalc(rf_df)
svm_df['Average']=aveCalc(svm_df)

## Random Forest Results

In [12]:
rf_df

,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,Average
FP,57.000000,68.000000,79.000000,69.00000,58.000000,80.000000,70.000000,62.000000,57.00000,70.000000,67.000000
FN,57.000000,68.000000,79.000000,69.00000,58.000000,80.000000,70.000000,62.000000,57.00000,70.000000,67.000000
TP,103.000000,92.000000,81.000000,91.00000,102.000000,80.000000,90.000000,98.000000,103.00000,89.000000,92.900000
TN,583.000000,572.000000,721.000000,731.00000,582.000000,720.000000,570.000000,578.000000,743.00000,725.000000,652.500000
Positive,160.000000,160.000000,160.000000,160.00000,160.000000,160.000000,160.000000,160.000000,160.00000,159.000000,159.900000
Negative,640.000000,640.000000,800.000000,800.00000,640.000000,800.000000,640.000000,640.000000,800.00000,795.000000,719.500000
TPR,0.643750,0.575000,0.506250,0.56875,0.637500,0.500000,0.562500,0.612500,0.64375,0.559748,0.580975
TNR,0.910937,0.893750,0.901250,0.91375,0.909375,0.900000,0.890625,0.903125,0.92875,0.911950,0.906351
FPR,0.089063,0.106250,0.098750,0.08625,0.090625,0.100000,0.109375,0.096875,0.07125,0.088050,0.093649
FNR,0.356250,0.425000,0.493750,0.43125,0.362500,0.500000,0.437500,0.387500,0.35625,0.440252,0.419025


# SVM Results

In [13]:
svm_df

,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,Average
FP,75.000000,69.000000,81.00000,82.000000,81.000000,101.000000,86.000000,82.000000,70.000000,83.000000,81.000000
FN,75.000000,69.000000,81.00000,82.000000,81.000000,101.000000,86.000000,82.000000,70.000000,83.000000,81.000000
TP,85.000000,91.000000,79.00000,78.000000,79.000000,59.000000,74.000000,78.000000,90.000000,76.000000,78.900000
TN,405.000000,571.000000,719.00000,718.000000,559.000000,699.000000,394.000000,558.000000,730.000000,712.000000,606.500000
Positive,160.000000,160.000000,160.00000,160.000000,160.000000,160.000000,160.000000,160.000000,160.000000,159.000000,159.900000
Negative,480.000000,640.000000,800.00000,800.000000,640.000000,800.000000,480.000000,640.000000,800.000000,795.000000,687.500000
TPR,0.531250,0.568750,0.49375,0.487500,0.493750,0.368750,0.462500,0.487500,0.562500,0.477987,0.493424
TNR,0.843750,0.892188,0.89875,0.897500,0.873437,0.873750,0.820833,0.871875,0.912500,0.895597,0.878018
FPR,0.156250,0.107813,0.10125,0.102500,0.126562,0.126250,0.179167,0.128125,0.087500,0.104403,0.121982
FNR,0.468750,0.431250,0.50625,0.512500,0.506250,0.631250,0.537500,0.512500,0.437500,0.522013,0.506576


# LSTM

In [14]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import metrics 

In [15]:
lstm_mod = Sequential()
lstm_mod.add(Dense(12, activation ='softmax', input_shape =(11, )))
lstm_mod.add(Dense(9, activation ='softmax'))
lstm_mod.add(Dense(1, activation ='sigmoid'))
lstm_mod.output_shape
lstm_mod.summary()
lstm_mod.get_config()
  
# List all weight tensors
lstm_mod.get_weights()
lstm_mod.compile(loss ='binary_crossentropy', 
  optimizer ='adamax', metrics = [metrics.categorical_accuracy])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                144       
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 117       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10        
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________


In [16]:
for train_ind , test_ind in kf.split(X):
    fc=fc+1
    cn= 'fold '+str(fc)
    X_train , X_test = X.iloc[train_ind,:],X.iloc[test_ind,:]
    y_train , y_test = y[train_ind] , y[test_ind]
    lstm_mod.fit(X_train, y_train, epochs = 3,batch_size = 32, verbose = 1,validation_split=0)
    y_pred = lstm_mod.predict(X_test)
    print(y_pred)

Epoch 1/3
45/45 [==============================] - 0s 614us/step - loss: -0.2400 - categorical_accuracy: 1.0000
Epoch 2/3
45/45 [==============================] - 0s 591us/step - loss: -0.8603 - categorical_accuracy: 1.0000
Epoch 3/3
45/45 [==============================] - 0s 591us/step - loss: -1.4370 - categorical_accuracy: 1.0000
[[0.62320244]
 [0.62320304]
 [0.6232028 ]
 [0.6232029 ]
 [0.62320244]
 [0.62320274]
 [0.6232022 ]
 [0.62320316]
 [0.6232026 ]
 [0.6232012 ]
 [0.6232011 ]
 [0.6232012 ]
 [0.62320244]
 [0.6232009 ]
 [0.62320304]
 [0.62320304]
 [0.62320304]
 [0.6232028 ]
 [0.6231999 ]
 [0.6232029 ]
 [0.62320304]
 [0.62320304]
 [0.6232002 ]
 [0.62311023]
 [0.62320304]
 [0.6232027 ]
 [0.6231941 ]
 [0.6232002 ]
 [0.623203  ]
 [0.62320244]
 [0.623201  ]
 [0.6232031 ]
 [0.62297773]
 [0.62320304]
 [0.62320065]
 [0.6232023 ]
 [0.62318844]
 [0.623203  ]
 [0.62320065]
 [0.6231885 ]
 [0.6231885 ]
 [0.62320304]
 [0.6232026 ]
 [0.62320244]
 [0.6231982 ]
 [0.6229508 ]
 [0.6232014 ]
 [0.62

 [0.81247056]]
Epoch 1/3
45/45 [==============================] - 0s 523us/step - loss: -6.8782 - categorical_accuracy: 1.0000
Epoch 2/3
45/45 [==============================] - 0s 500us/step - loss: -7.3558 - categorical_accuracy: 1.0000
Epoch 3/3
45/45 [==============================] - 0s 500us/step - loss: -7.8298 - categorical_accuracy: 1.0000
[[0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343   ]
 [0.8534306 ]
 [0.8534303 ]
 [0.85343087]
 [0.85343087]
 [0.8534307 ]
 [0.85343087]
 [0.85343087]
 [0.8534308 ]
 [0.85343045]
 [0.85343087]
 [0.8534307 ]
 [0.85343087]
 [0.8534288 ]
 [0.8534302 ]
 [0.8534288 ]
 [0.85343087]
 [0.8534306 ]
 [0.8534302 ]
 [0.85343087]
 [0.85343087]
 [0.85343045]
 [0.8534307 ]
 [0.85343087]
 [0.8534307 ]
 [0.85343087]
 [0.8534299 ]
 [0.85343087]
 [0.8534299 ]
 [0.85343075]
 [0.85343087]
 [0.85343075]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343087]
 [0.85343045]
 [0.85343087]
 [0.85343045]
 [0.8

 [0.9309891 ]]
Epoch 1/3
45/45 [==============================] - 0s 523us/step - loss: -12.2519 - categorical_accuracy: 1.0000
Epoch 2/3
45/45 [==============================] - 0s 500us/step - loss: -12.6823 - categorical_accuracy: 1.0000
Epoch 3/3
45/45 [==============================] - 0s 500us/step - loss: -13.1056 - categorical_accuracy: 1.0000
[[0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.94649   ]
 [0.9465077 ]
 [0.9465077 ]
 [0.94650745]
 [0.9465077 ]
 [0.9465077 ]
 [0.94650745]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465059 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465076 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465076 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [0.9465077 ]
 [